In [65]:
'''
Requirements:
 - Caffe (script to install Caffe and pycaffe on a new Ubuntu 14.04 LTS x64 or Ubuntu 14.10 x64. 
   CPU only, multi-threaded Caffe. http://stackoverflow.com/a/31396229/395857)
 - sudo pip install pydot
 - sudo apt-get install -y graphviz

Interesting resources on Caffe:
 - https://github.com/BVLC/caffe/tree/master/examples
 - http://nbviewer.ipython.org/github/joyofdata/joyofdata-articles\
 /blob/master/deeplearning-with-caffe/\
 Neural-Networks-with-Caffe-on-the-GPU.ipynb
'''

import subprocess
import platform
import copy
import sys
import os

# from sklearn.datasets import load_iris
import sklearn.metrics
import numpy as np
from sklearn.cross_validation import StratifiedShuffleSplit
import matplotlib.pyplot as plt
import h5py
import caffe
import caffe.draw
import google.protobuf 

# Globals
train_data, train_labels, test_data, test_labels = [], [], [], []

def load_data():
    '''
    Load Sample for Forward Pass from Toy Car Data set
    '''
    start_states, controls, durations, end_states = [], [], [], []

    with open('data_output_50Hz.txt', 'r') as infile:
        data = infile.readlines()

        idx, i = 0, 0

        for line in data:
            # Stop at end of file
            if line == '':
                break

            # Reset and continue at Trajectory break
            if len(line) == 1:
                start_states.pop()
                i = 0
                if idx > 1000000:
                    break
                continue
                
            # Split Values in line and append to individual lists
            vals = line.split(',')
            if i % 3 == 0:
                start_states.append([float(val) for val in vals])
                if i != 0:
                    end_states.append([float(val) for val in vals])
                    idx += 1
            elif i % 3 == 1:
                controls.append([float(val) for val in vals])
            elif i % 3 == 2:
                durations.append([float(val) for val in vals])
                
            i += 1
            
    X = np.concatenate((start_states, controls, durations), axis=1)
    start_states = np.asarray(start_states, dtype=np.float32)
    end_states = np.asarray(end_states, dtype=np.float32)
    
    X, meanx, minx, maxx = normalize_data(X)
    y = normalize_labels(start_states, end_states)

    # Shuffle the data around and split 3M training, ~750k validation
    indices = np.random.permutation(X.shape[0])
    training_idx = indices[:900000]
    
    train_X = X[training_idx, :]
    train_y = y[training_idx, :]

    test_X = X[900000:1000000, :]
    test_y = y[900000:1000000, :]

    return train_X, train_y, test_X, test_y, meanx, minx, maxx


def write_binaryproto(data, string):
    blob = caffe.proto.caffe_pb2.BlobProto()
    blob.channels = data.shape[0]
    blob.data.extend(data.astype(float).flat)
    binaryproto_file = open('toycar_' + string + '.binaryproto', 'wb')
    binaryproto_file.write(blob.SerializeToString())
    binaryproto_file.close()


def save_binaryproto(data, ftype='mean'):
    '''
    Take the mean values of the raw data and store them as binaryproto type
    In order to use them later for deploy normalization
    '''
    # Convert to 32bit float
    data = np.array(data, dtype=np.float32)

    # Set project home dir 
    PROJECT_HOME = os.path.abspath('.')

    # Initialize blob to store serialized means
    blob = caffe.proto.caffe_pb2.BlobProto()

    # Custom dimensions for blob for this project
    blob.num = 1
    blob.channels = data.shape[0]
    blob.height = 1
    blob.width = 1
    
    # Reshape data and copy into blob\n",
    blob.data.extend(data.astype(float).flat)
    
    # Write file
    binaryproto_file = open(PROJECT_HOME + '/toycar_' + ftype + '.binaryproto', 'wb')
    binaryproto_file.write(blob.SerializeToString())
    binaryproto_file.close()



def normalize_labels(start_states, end_states):
    '''
    Normalize end states such that positional coordinates e.g. (x,y)
    are now represented by delta_(x,y)
    '''
    y = end_states
    y[:, 0] = end_states[:, 0] - start_states[:, 0]
    y[:, 1] = end_states[:, 1] - start_states[:, 1]

    return y


def unnormalize_data(data, meanx, minx, maxx):
    desired_min = -1
    desired_max = 1
    desired_rng = desired_max - desired_min
    
    data = data - desired_min
    for i in range(0, data.shape[1]):
        data[:,i] = data[:,i] * (maxx[:,i] - minx[:,i]) \
                    / desired_rng + minx[:,i] + meanx[:,i]
        
    return data


def normalize_data(data):
    '''
    Normalize data to zero mean on [-1,1] interval for all dimensions
    '''
    X_mean = np.mean(data, axis=0)
    
    # do not substract duration
    X_mean[7] = 0
    
    # Mean Shift
    data_t = data - X_mean
    
    # Find bounds, define desired bounds
    X_min = np.min(data_t, axis=0)
    X_max = np.max(data_t, axis=0)

    # Write 'em Out
    save_binaryproto(X_mean, ftype="mean")
    save_binaryproto(X_min, ftype="min")
    save_binaryproto(X_max, ftype="max")

    desiredMin = -1
    desiredMax = 1
    
    # Normalize 
    for i in range(0, 7):
        data_t[:, i] = (data_t[:, i] - X_min[i]) * (desiredMax - desiredMin)\
            / (X_max[i] - X_min[i]) + desiredMin

    return data_t, X_mean, X_min, X_max


# def normalize_data(data, X_mean, X_min, X_max):
#     '''
#     Normalize data to zero mean on [-1,1] interval for all dimensions
#     '''
# #     X_mean = np.mean(data, axis=0)
    
#     # do not substract duration
# #     X_mean[7] = 0
    
#     # Mean Shift
#     data_t = data - X_mean
    
#     # Find bounds, define desired bounds
#     X_min = np.min(data_t, axis=0)
#     X_max = np.max(data_t, axis=0)

#     # Write 'em Out
# #     save_binaryproto(X_mean, ftype="mean")
# #     save_binaryproto(X_min, ftype="min")
# #     save_binaryproto(X_max, ftype="max")

#     desiredMin = -1
#     desiredMax = 1
    
#     # Normalize 
#     for i in range(0, 7):
#         data_t[:, i] = (data_t[:, i] - X_min[i]) * (desiredMax - desiredMin)\
#             / (X_max[i] - X_min[i]) + desiredMin

#     return data_t, X_mean, X_min, X_max

def normalize_test_data(data, meanx, minx, maxx):
    '''
    Normalize data to zero mean on [-1,1] interval for all dimensions
    '''
    
    # Mean Shift
#     print "data: ", data.shape
#     print "mean: ", meanx.shape
#     print "min: ", minx.shape
#     print "max: ", maxx.shape
    
#     print "DATA", data
#     print "MEAN", meanx
    
    data_t = data - meanx
    
#     print "DATA - MEAN: ", data_t
    
    desiredMin = -1
    desiredMax = 1
    
    # Normalize 
    for i in range(0, data.shape[1]):
        data_t[:, i] = (data_t[:, i] - minx[:, i]) * (desiredMax - desiredMin)\
            / (maxx[:, i] - minx[:, i]) + desiredMin

    return data_t


def save_data_as_hdf5(hdf5_data_filename, data, labels):
    '''
    HDF5 is one of the data formats Caffe accepts
    '''
    with h5py.File(hdf5_data_filename, 'w') as f:
        f['data'] = data.astype(np.float32)
        f['label'] = labels.astype(np.float32)


def train(solver_prototxt_filename):
    '''
    Train the ANN
    '''
    # caffe.set_mode_gpu()
    solver = caffe.get_solver(solver_prototxt_filename)
    solver.solve()


def print_network_parameters(net):
    '''
    Print the parameters of the network
    '''
    print(net)
    print('net.inputs: {0}'.format(net.inputs))
    print('net.outputs: {0}'.format(net.outputs))
    print('net.blobs: {0}'.format(net.blobs))
    print('net.params: {0}'.format(net.params))    


def get_predicted_output(deploy_prototxt_filename, 
                         caffemodel_filename, input, net=None):
    '''
    Get the predicted output, i.e. perform a forward pass
    '''
    if net is None:
        net = caffe.Net(deploy_prototxt_filename, 
                        caffemodel_filename, caffe.TEST)
    
#     print "Input: "
#     print input 
    out = net.forward(data=input)
    return out[net.outputs[0]]


def print_network(prototxt_filename, caffemodel_filename):
    '''
    Draw the ANN architecture
    '''
    _net = caffe.proto.caffe_pb2.NetParameter()
    f = open(prototxt_filename)
    google.protobuf.text_format.Merge(f.read(), _net)
    caffe.draw.draw_net_to_file(_net, prototxt_filename + '.png')
    print('Draw ANN done!')


def print_network_weights(prototxt_filename, caffemodel_filename):
    '''
    For each ANN layer, print weight heatmap and weight histogram 
    '''
    net = caffe.Net(prototxt_filename, caffemodel_filename, caffe.TEST)
    for layer_name in net.params: 
        # weights heatmap 
        arr = net.params[layer_name][0].data
        plt.clf()
        fig = plt.figure(figsize=(10, 10))
        ax = fig.add_subplot(111)
        cax = ax.matshow(arr, interpolation='none')
        fig.colorbar(cax, orientation="horizontal")
        plt.savefig('{0}_weights_{1}.png'.format(caffemodel_filename, 
                                                 layer_name), 
                    dpi=100, format='png', bbox_inches='tight')
        plt.close()

        # weights histogram  
        plt.clf()
        plt.hist(arr.tolist(), bins=20)
        # savefig: use format='svg' or 'pdf' for vectorial pictures
        plt.savefig('{0}_weights_hist_{1}.png'.format(caffemodel_filename, 
                                                      layer_name), dpi=100, 
                    format='png', 
                    bbox_inches='tight')  
        plt.close()


def get_predicted_outputs(deploy_prototxt_filename, 
                          caffemodel_filename, inputs):
    '''
    Get several predicted outputs
    '''
    outputs = []
    net = caffe.Net(deploy_prototxt_filename, caffemodel_filename, caffe.TRAIN)
    outputs.append(copy.deepcopy(get_predicted_output(deploy_prototxt_filename, 
                                                      caffemodel_filename, 
                                                      inputs, net)))
    return outputs    


def get_accuracy(true_outputs, predicted_outputs):
    '''

    '''
    number_of_samples = true_outputs.shape[0]
    number_of_outputs = true_outputs.shape[1]
    threshold = 0.0  # 0 if SigmoidCrossEntropyLoss ; 0.5 if EuclideanLoss
    for output_number in range(number_of_outputs):
        predicted_output_binary = []
        for sample_number in range(number_of_samples):
            # print(predicted_outputs)
            # print(predicted_outputs[sample_number][output_number])            
            if predicted_outputs[sample_number][0][output_number] < threshold:
                predicted_output = 0
            else:
                predicted_output = 1
            predicted_output_binary.append(predicted_output)

        print('accuracy: {0}'.format(sklearn.metrics.accuracy_score(
                                     true_outputs[:, output_number], 
                                     predicted_output_binary)))
        print(sklearn.metrics.confusion_matrix(true_outputs[:, output_number], 
                                               predicted_output_binary))


def training(model_iter):
    '''
    Performs Training of the specified network and outputs PNG images
    showing resulting learned weights and histograms of weights
    '''
    # Set parameters
    solver_prototxt_filename = 'toycar_solver.prototxt'
    train_test_prototxt_filename = 'toycar_2fc_hdf5.prototxt'
    caffemodel_filename = '2fc_iter_' + str(model_iter) + '.caffemodel' 

    # Train network
    train(solver_prototxt_filename)

    # Print network
    print_network(train_test_prototxt_filename, caffemodel_filename)
    print_network_weights(train_test_prototxt_filename, caffemodel_filename)


def testing(deploy_prototxt_filename, caffemodel_filename, inputs, labels):
    '''
    Performs Testing of the specified network
    '''    
    # Compute performance metrics
    outputs = get_predicted_outputs(deploy_prototxt_filename, 
                                    caffemodel_filename, inputs)
    
#     print 'predictions: '
#     print outputs[0]
    
#     print 'ground truths: '
#     print labels
    
    return outputs[0]
    
    
def euclidean_loss(pred, labels):
    '''
    Hand Calculate the Euclidean Loss to Compare with Model Output
    '''
    result = labels-pred
    size = pred.shape[0]
    
    loss = np.sum(np.square(result), axis=1)
    loss = np.sum(loss, axis=0) / (2 * size)
    
    print "Euclidean Loss: ", loss


def main(arg="x"):

    if arg.lower() == "train":
        train_data, train_labels, test_data, test_labels = load_data()

        # save_data_as_hdf5('toycar_hdf5_data_random_norm11_train.hdf5', 
        #                   train_data, train_labels)
        # save_data_as_hdf5('toycar_hdf5_data_random_norm11_test.hdf5', 
        #                   test_data, test_labels)
        
        solver_name = "toycar_solver.prototxt"
        training(20000)

    elif arg.lower() == "test":
        
        train_data, train_labels, test_data, test_labels, meanx, minx, maxx = load_data()

        pred = testing('toycar_2fc_deploy.prototxt', 
               '2fc_iter_100000.caffemodel', 
               test_data[0:1, :], test_labels[0:1, :])
        print "Input: ", test_data[0:1, :]
#             pred[0, 0] = pred[0, 0] + test_data[0, 0]
#             pred[0, 1] = pred[0, 1] + test_data[0, 1]
        print "Pred: ", pred
        print "Label: ", test_labels[0:1, :]
        print "Loss: ", euclidean_loss(pred, test_labels[0:1, :])


        for i in range(1, 100):
            
#             print pred.shape
#             print test_data[i:i+1,5:8].shape
        
#             print "test: ", test_data[i-1,0:2].shape
#             print "meanx: ", np.asarray([meanx[0:2]]).shape
#             print "minx: ", np.asarray([minx[0:2]]).shape
#             print "maxx: ", np.asarray([maxx[0:2]]).shape
            unnorm_state = unnormalize_data(test_data[i-1:i,0:2], np.asarray([meanx[0:2]])
                                            , np.asarray([minx[0:2]]),np.asarray([maxx[0:2]]))
        
            pred[0, 0] = pred[0, 0] + unnorm_state[0,0]
            pred[0, 1] = pred[0, 1] + unnorm_state[0,1]
            
            
            pred = normalize_test_data(pred, 
                                       np.asarray([meanx[0:5]]), 
                                       np.asarray([minx[0:5]]), 
                                       np.asarray([maxx[0:5]]))
            
            inpt = np.asarray([np.concatenate((pred[0,:], test_data[i, 5:8]), axis=0)])
            
            print "Test Data: ", test_data[i,:]
            print "Input: ", inpt
            pred = testing('toycar_2fc_deploy.prototxt', 
                           '2fc_iter_100000.caffemodel', 
                           inpt,  test_labels[i:i+1, :])
            
            print "Pred: ", pred
            print "Label: ", test_labels[i:i+1, :]
            print "Loss: ", euclidean_loss(pred, test_labels[i:i+1, :])
            print "\n"
    else:
        train_data, train_labels, test_data, test_labels = load_data()


    # result = pred - test_labels[:1000, :]
    # print np.sqrt(np.sum(np.square(result), axis=1)).shape

    
    

In [66]:
# main()
main("test")

Input:  [[-0.52753878 -0.09802863  0.13742372 -0.54690999  0.16745029 -0.37171077
   0.50107972  0.02      ]]
Pred:  [[-0.00133245 -0.0015051   0.13568482 -0.09937215  0.09792733]]
Label:  [[-0.00196064 -0.00026777  0.13562027 -0.10375793  0.09954461]]
Loss:  Euclidean Loss:  1.18902407849e-05
None
Test Data:  [-0.52800329 -0.09813368  0.13728903 -0.55064514  0.17081498 -0.37171077
  0.50107972  0.02      ]
Input:  [[-0.52785446 -0.0986191   0.13733354 -0.54844139  0.16810029 -0.37171077
   0.50107972  0.02      ]]
Pred:  [[-0.00139116 -0.00151112  0.13554779 -0.10242885  0.09835221]]
Label:  [[-0.00210685 -0.00028731  0.13540609 -0.11119144  0.10154913]]
Loss:  Euclidean Loss:  4.45166588179e-05
None


Test Data:  [-0.52850244 -0.09824639  0.13714132 -0.55438028  0.17417968 -0.37171077
  0.50107972  0.02      ]
Input:  [[-0.52833288 -0.09872651  0.13723904 -0.54997731  0.16881347 -0.37171077
   0.50107972  0.02      ]]
Pred:  [[-0.00145112 -0.00151574  0.13540891 -0.10551107  0.098787

In [29]:
train_data, train_labels, test_data, test_labels, meanx, minx, maxx = load_data()
check = np.asarray([[0, 0, 0.13562027, -0.10375793,  0.09954461]])
res = normalize_test_data(check,
                          np.asarray([meanx[0:5]]), 
                           np.asarray([minx[0:5]]), 
                           np.asarray([maxx[0:5]]))

In [33]:
i=0
for x in test_data:
    if i < 100:
        print test_data[i,:]
        i+=1

[-0.52753878 -0.09802863  0.13742372 -0.54690999  0.16745029 -0.37171077
  0.50107972  0.02      ]
[-0.52800329 -0.09813368  0.13728903 -0.55064514  0.17081498 -0.37171077
  0.50107972  0.02      ]
[-0.52850244 -0.09824639  0.13714132 -0.55438028  0.17417968 -0.37171077
  0.50107972  0.02      ]
[-0.52903622 -0.09836672  0.1369802  -0.55811543  0.17754437 -0.37171077
  0.50107972  0.02      ]
[-0.52960462 -0.09849463  0.13680525 -0.56185058  0.18090906 -0.37171077
  0.50107972  0.02      ]
[-0.53020763 -0.09863006  0.13661607 -0.56558573  0.18427376 -0.37171077
  0.50107972  0.02      ]
[-0.53084522 -0.09877295  0.13641224 -0.56932087  0.18763845 -0.37171077
  0.50107972  0.02      ]
[-0.5315174  -0.09892324  0.13619336 -0.57305602  0.19100315 -0.37171077
  0.50107972  0.02      ]
[-0.53222415 -0.09908088  0.13595903 -0.57679117  0.19436784 -0.37171077
  0.50107972  0.02      ]
[-0.53296544 -0.09924579  0.13570882 -0.58052632  0.19773254 -0.37171077
  0.50107972  0.02      ]
[-0.533741

In [35]:
print unnormalize_data(test_data[1:2,:], meanx, minx, maxx)
print test_labels[0,:]

[[-0.95341616  0.86100332  1.85468962 -1.04898425  1.99619798 -0.29372734
   3.39019925  1.35962904]]
[-0.00196064 -0.00026777  0.13562027 -0.10375793  0.09954461]


In [57]:
a = np.asarray([[1, 2, 3, 4, 5]])
print meanx
print minx,maxx
norm_a = normalize_test_data(a, np.asarray([meanx[0:5]]), np.asarray([minx]), np.asarray([maxx]))
print 'norm_a:', norm_a
unnorm_a = unnormalize_data(norm_a, np.asarray([meanx[0:5]]), np.asarray([minx[0:5]]), 
                            np.asarray([maxx[0:5]]))
print 'unnorm_a: ',unnorm_a

[ 0.29162416  0.00804919  0.03366669  0.27424605  0.00841767  0.05690521
  0.00058458  0.        ]
[-3.23727242 -2.3730294  -1.54712097 -1.27228815 -0.6063877  -1.05684408
 -0.20053342  0.02      ] [ 5.20444778  2.72492031  1.35288862  2.70801442  0.58511506  0.94307371
  0.19941189  0.02      ]
norm_a: [[-0.06520279  0.71244538  2.1127168   1.51138802  8.39648689]]
unnorm_a:  [[ 1.  2.  3.  4.  5.]]


In [25]:

blob = caffe.proto.caffe_pb2.BlobProto()
data = open( "toycar_mean.binaryproto" , 'rb' ).read()
blob.ParseFromString(data)
# arr = np.array( caffe.io.blobproto_to_array(blob) )
# out = arr[0]
print blob.data

data = open( "../prx_ws/src/prx_learn/data/toy_car/toy_car_mean.binaryproto" , 'rb' ).read()
blob.ParseFromString(data)
print blob.data


[0.29162415862083435, 0.008049188181757927, 0.033666692674160004, 0.27424606680870056, 0.008417674340307713, 0.05690521001815796, 0.0005845790728926659, 0.0]
[0.6776861548423767, -0.06710384786128998, -0.05981121584773064, 0.4085806608200073, -0.004740194883197546, 0.04174768552184105, 0.0003070685488637537, 0.0]
